# Pre-requisits

First make sure that all the dependencies are available


In [1]:
# brew cask install google-cloud-sdk helm kubectl git
brew install terraform helm kubectl git
gcloud components install gke-gcloud-auth-plugin -y

terraform -install-autocomplete

Running `brew update --auto-update`...
==> Auto-updated Homebrew!
Updated 3 taps (homebrew/core, homebrew/cask and homebrew/services).
==> New Formulae
burst                                    ripsecrets
purescript-language-server               swiftdraw
rdb
==> New Casks
decentr             filen               fresh               goxel

You have 6 outdated formulae and 1 outdated cask installed.
You can upgrade them with brew upgrade
or list them with brew outdated.

To reinstall 1.2.7, run:
  brew reinstall terraform
To reinstall 3.9.3, run:
  brew reinstall helm
kubernetes-cli 1.24.3 is already installed but outdated (so it will be upgraded).
To reinstall 2.37.2, run:
  brew reinstall git
==> Downloading https://ghcr.io/v2/homebrew/core/kubernetes-cli/manifests/1.24.4
######################################################################## 100.0%
==> Downloading https://ghcr.io/v2/homebrew/core/kubernetes-cli/blobs/sha256:0ec
==> Downloading from https://pkg-containers.githubusercon

: 1

Continue by adding the required terraform repositoreis

Setup project with gcloud, this project ID is EXTREMELEY IMPORANT for the rest of the tutorial. Make sure to update the value (if YOU CHOOSE to change it
to something, as well in the variables.tf file).


In [4]:
ACCOUNT_ID="terraform-iam-service-account"
PROJECT_ID="qpecs-fltk-2022"
PRIVILEGED_ACCOUNT_ID="${ACCOUNT_ID}@${PROJECT_ID}.iam.gserviceaccount.com"
CLUSTER_NAME="freddie-testbed-cluster"
REGION="us-central1"

gcloud projects create $PROJECT_ID --set-as-default
gcloud beta billing accounts list # Copy the Account ID of the account

ERROR: (gcloud.projects.create) Project creation failed. The project ID you specified is already in use by another project. Please try an alternative ID.
ACCOUNT_ID            NAME                           OPEN   MASTER_ACCOUNT_ID
010CFE-95784F-3A6A83  Billing Account for Education  False
011C4F-1F8FD5-BDEDDD  Billing Account for Education  False
015594-41687F-092941  Billing Account for Education  True
01610A-173A5E-0FC84C  Billing Account for Education  False
019AAC-7C541E-616936  Billing Account for Education  False


In [2]:
echo $PRIVILEGED_ACCOUNT_ID

terraform-iam-service-account@terraform-iam-service-account.iam.gserviceaccount.com


Setup billing and enable services create service account. This way we don't have to create key files.

In [24]:
# Setup billing to project
gcloud beta billing projects link $PROJECT_ID --billing-account 015594-41687F-092941
# Enable services now billing is enabled
gcloud services enable compute container --project $PROJECT_ID

billingAccountName: billingAccounts/015594-41687F-092941
billingEnabled: true
name: projects/qpecs-fltk-2022/billingInfo
projectId: qpecs-fltk-2022
Operation "operations/acat.p2-507430712695-6f66cea7-2ce4-4190-b9eb-4a0e403e4744" finished successfully.


Create service account that has the minimum set of permissions for creating and managing a cluster.

In [5]:
function enable_gcp_role () {
  gcloud projects  add-iam-policy-binding \
    $PROJECT_ID \
    --member="serviceAccount:$PRIVILEGED_ACCOUNT_ID" \
    --role="roles/$1"
}

gcloud iam service-accounts create $ACCOUNT_ID --display-name="Terraform service account" --project ${PROJECT_ID}

enable_gcp_role "compute.viewer"
enable_gcp_role "compute.securityAdmin"
enable_gcp_role "container.clusterViewer"
enable_gcp_role "container.clusterAdmin"
enable_gcp_role "container.developer"
enable_gcp_role "iam.serviceAccountAdmin"
enable_gcp_role "iam.serviceAccountUser"
enable_gcp_role "compute.networkAdmin"

ERROR: (gcloud.iam.service-accounts.create) Resource in projects [qpecs-fltk-2022] is the subject of a conflict: Service account terraform-iam-service-account already exists within project projects/qpecs-fltk-2022.
- '@type': type.googleapis.com/google.rpc.ResourceInfo
  resourceName: projects/qpecs-fltk-2022/serviceAccounts/terraform-iam-service-account@qpecs-fltk-2022.iam.gserviceaccount.com
Updated IAM policy for project [qpecs-fltk-2022].
bindings:
- members:
  - serviceAccount:service-507430712695@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:terraform-iam-service-account@qpecs-fltk-2022.iam.gserviceaccount.com
  role: roles/compute.viewer
- members:
  - serviceAccount:service-507430712695@container-engine-robot.iam.gserviceaccount.com
  role: roles/container.serviceAgent
- members:
  - serviceAccount:service-507430712695@containerregistry.iam.gserviceaccount.com
  role: roles/containerregistry.ServiceAgent
- members:
  - s

Now we enable impersonalisation, to allow the main account of the project to make use of the service account. Thereby reducing
potential security issues

Assign your `google_account` mail to the `OWNER_MAIL` variable, and run the command box below.

In [6]:
OWNER_MAIL="jargsnork@gmail.com"
gcloud iam service-accounts add-iam-policy-binding $PRIVILEGED_ACCOUNT_ID \
 --member="user:$OWNER_MAIL" \
 --role=roles/iam.serviceAccountTokenCreator \
 --project $PROJECT_ID

# By default we can now use impersionation with the service account. Additionally you can add users to interact with the
# service account.
# gcloud iam service-accounts add-iam-policy-binding \
#     $PRIVILEGED_ACCOUNT_MAIL \
#     --member='user:example.user.goes.here@<domain@>.com' --role='roles/editor'


Updated IAM policy for serviceAccount [terraform-iam-service-account@qpecs-fltk-2022.iam.gserviceaccount.com].
bindings:
- members:
  - user:jargsnork@gmail.com
  role: roles/iam.serviceAccountTokenCreator
etag: BwXm1QDSgg0=
version: 1


In [7]:
cd terraform-gke

In [8]:
echo $PWD

/Users/jeroen/fltk-project/fltk-testbed/jupyter/terraform-gke


In [9]:
terraform init -reconfigure

Initializing modules...

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/google from the dependency lock file
- Reusing previous version of hashicorp/kubernetes from the dependency lock file
- Reusing previous version of hashicorp/google-beta from the dependency lock file
- Reusing previous version of hashicorp/random from the dependency lock file
- Using previously-installed hashicorp/google v4.32.0
- Using previously-installed hashicorp/kubernetes v2.12.1
- Using previously-installed hashicorp/google-beta v4.32.0
- Using previously-installed hashicorp/random v3.3.2

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, oth

In [12]:
terraform plan

data.google_service_account_access_token.default: Reading...
data.google_service_account_access_token.default: Read complete after 0s [id=projects/-/serviceAccounts/terraform-iam-service-account@qpecs-fltk-2022.iam.gserviceaccount.com]
data.google_client_config.default: Reading...
data.google_client_config.default: Read complete after 0s [id=projects/qpecs-fltk-2022/regions//zones/]
module.gke.data.google_container_engine_versions.region: Reading...
module.gke.data.google_compute_zones.available: Reading...
module.gke.data.google_container_engine_versions.region: Read complete after 0s [id=2022-08-22 14:12:19.966472 +0000 UTC]
module.gke.data.google_compute_zones.available: Read complete after 0s [id=projects/qpecs-fltk-2022/regions/us-central1]
module.gke.data.google_container_engine_versions.zone: Reading...
module.gke.data.google_container_engine_versions.zone: Read complete after 0s [id=2022-08-22 14:12:20.347245 +0000 UTC]

Terraform used the selected providers to generate the fol

In [13]:
terraform apply -auto-approve

data.google_service_account_access_token.default: Reading...
data.google_service_account_access_token.default: Read complete after 0s [id=projects/-/serviceAccounts/terraform-iam-service-account@qpecs-fltk-2022.iam.gserviceaccount.com]
module.gke.data.google_compute_zones.available: Reading...
module.gke.data.google_container_engine_versions.region: Reading...
data.google_client_config.default: Reading...
data.google_client_config.default: Read complete after 0s [id=projects/qpecs-fltk-2022/regions//zones/]
module.gke.data.google_container_engine_versions.region: Read complete after 0s [id=2022-08-22 14:12:56.882912 +0000 UTC]
module.gke.data.google_compute_zones.available: Read complete after 0s [id=projects/qpecs-fltk-2022/regions/us-central1]
module.gke.data.google_container_engine_versions.zone: Reading...
module.gke.data.google_container_engine_versions.zone: Read complete after 0s [id=2022-08-22 14:12:57.324072 +0000 UTC]

Terraform used the selected providers to generate the fol

In [15]:
# Add credentials for interacting with cluster via kubectl
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID
cd ../terraform-kubeflow

Fetching cluster endpoint and auth data.
kubeconfig entry generated for freddie-testbed-cluster.


In [16]:
terraform init -reconfigure

Initializing modules...

Initializing the backend...

Initializing provider plugins...
- Reusing previous version of hashicorp/kubernetes from the dependency lock file
- Reusing previous version of hashicorp/google from the dependency lock file
- Reusing previous version of hashicorp/time from the dependency lock file
- Reusing previous version of hashicorp/template from the dependency lock file
- Reusing previous version of kbst/kustomization from the dependency lock file
- Reusing previous version of gavinbunney/kubectl from the dependency lock file
- Using previously-installed hashicorp/kubernetes v2.12.1
- Using previously-installed hashicorp/google v4.32.0
- Using previously-installed hashicorp/time v0.8.0
- Using previously-installed hashicorp/template v2.2.0
- Using previously-installed kbst/kustomization v0.9.0
- Using previously-installed gavinbunney/kubectl v1.14.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform 

In [17]:
terraform plan

module.kubeflow.data.template_file.config_yaml: Reading...
module.kubeflow.data.template_file.config_yaml: Read complete after 0s [id=e70434e7cbdc7b7b42ddd875f3c0aa739f8612543152222cc4e6bfae9394b994]
module.kubeflow.data.kustomization_build.kserve-web-app: Reading...
module.kubeflow.data.kustomization_overlay.knative-eventing: Reading...
module.kubeflow.data.kustomization_overlay.istio-ingress: Reading...
module.kubeflow.data.kustomization_build.istio-resources: Reading...
module.kubeflow.data.kustomization_build.letsencrypt-cluster-resources: Reading...
module.kubeflow.data.kustomization_build.volumes: Reading...
module.kubeflow.data.kustomization_build.profiles: Reading...
module.kubeflow.data.kustomization_build.katib: Reading...
module.kubeflow.data.kustomization_overlay.user-namespace: Reading...
module.kubeflow.data.kustomization_build.istio-namespace: Reading...
module.kubeflow.data.kustomization_build.kserve-web-app: Read complete after 2s [id=4aaa39409dbff6d5a30d26b901e1b3d2ba

In [ ]:
terraform apply

  Enter a value: 
╷
│ Error: error asking for approval: interrupted
│ 
│ 
╵

Interrupt received.
Please wait for Terraform to exit or data loss may occur.
Gracefully shutting down...

module.kubeflow.data.template_file.config_yaml: Reading...
module.kubeflow.data.template_file.config_yaml: Read complete after 0s [id=e70434e7cbdc7b7b42ddd875f3c0aa739f8612543152222cc4e6bfae9394b994]
module.kubeflow.data.kustomization_build.tensorboard-ui: Reading...
module.kubeflow.data.kustomization_build.katib: Reading...
module.kubeflow.data.kustomization_build.letsencrypt-cluster-resources: Reading...
module.kubeflow.data.kustomization_build.webhook: Reading...
module.kubeflow.data.kustomization_overlay.user-namespace: Reading...
module.kubeflow.data.kustomization_overlay.pipelines: Reading...
module.kubeflow.data.kustomization_build.profiles: Reading...
module.kubeflow.data.kustomization_build.istio-base: Reading...
module.kubeflow.data.kustomization_overlay.pipelines-metacontroller: Reading...
modu